In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os

In [3]:
import Misc.env_vars as ev
from Logic.base_queries import run_stmt
import Misc.env_vars as ev
from Logic import base_queries as bq

Found File.
Loaded Env from File.
5432
The port is  5432


In [4]:
from Models.models import (od_dq_base, dq_agg_view, dq_agg_sum, dq_col_sum, dq_dim_order_status, dq_dim_sellers, ds_data_sanity, dq_agg_order_stats,
ds_last_run_date)

In [21]:
from models import engine

In [5]:
from sqlalchemy import Select, func, or_, desc, and_

## Top Cards 

### Total Orders with Delta

In [6]:
dt_rng = bq.get_date_range()

max_date = dt_rng[1]

In [7]:
prev_dt = max_date - timedelta(days=1)

In [8]:
# seller = "agg.dominos.co.in"

In [9]:
prev_dt

datetime.date(2024, 6, 24)

In [10]:
st_dt = datetime.strptime("2024-08-10","%Y-%m-%d")

In [11]:
seller = None

In [30]:
def query_data_variance_report() -> pd.DataFrame:
    # for athena varience OD
    source_data_query = f"""
        SELECT
            month,
            sub_domain_name as sub_domain,
            dashboard,
            distinct_order_count as doc,
            distinct_order_count_till_last_run_date as doctlrd,
            run_date, last_run_date
        FROM
        {ev.PG_SCHEMA}.{ev.DS_TABLE} where month >= '{ev.DS_START_DATE}' order by run_date
    """
    df = pd.read_sql_query(source_data_query, engine)

    date_pairs = df[['run_date', 'last_run_date']].drop_duplicates().values.tolist()
    dataframes = []

    for run_date, last_run_date in date_pairs:

        if not run_date or not last_run_date:
            continue

        athena_od_varience_query = f"""
            WITH 
                base_data AS (
                    SELECT
                        month,
                        sub_domain_name as sub_domain,
                        dashboard,
                        distinct_order_count as doc,
                        distinct_order_count_till_last_run_date as doctlrd,
                        run_date, last_run_date

                    FROM
                    {ev.PG_SCHEMA}.{ev.DS_TABLE}  where run_date in ('{run_date}', '{last_run_date}')
                    and month >= '{ev.DS_START_DATE}'
                ),
                pivot_data AS (
                    SELECT
                        month,
                        (MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'B2C' AND run_date = '{run_date}' THEN doctlrd ELSE 0 END) -
                        MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'B2C' AND run_date = '{last_run_date}' THEN doc ELSE 0 END))
                        AS OD_B2C,

                        (MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'B2B' AND run_date = '{run_date}' THEN doctlrd ELSE 0 END) -
                        MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'B2B' AND run_date = '{last_run_date}' THEN doc ELSE 0 END))
                        AS OD_B2B,

                        (MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'Voucher' AND run_date = '{run_date}' THEN doctlrd ELSE 0 END) -
                        MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'Voucher' AND run_date = '{last_run_date}' THEN doc ELSE 0 END))
                        AS OD_GV,

                        (MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'Logistics' AND run_date = '{run_date}' THEN doctlrd ELSE 0 END) -
                        MAX(CASE WHEN dashboard = 'OD_with_base_query' AND sub_domain = 'Logistics' AND run_date = '{last_run_date}' THEN doc ELSE 0 END))
                        AS OD_Logistics

                    FROM
                        base_data
                    GROUP BY
                        month
                )
                SELECT
                    month,
                    OD_B2C as "OD_B2C",
                    OD_B2B as "OD_B2B",
                    OD_GV as "OD_GV",
                    OD_Logistics as OD_Logistics,
                    '{run_date} - {last_run_date}' as run_date_diff
                FROM
                    pivot_data
                ORDER BY
                    month DESC;

            """

        df = pd.read_sql(athena_od_varience_query, engine)

        # Append the resulting DataFrame to the list
        dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)

    return combined_df

In [31]:
df = query_data_variance_report()

In [32]:
df

,month,OD_B2C,OD_B2B,OD_GV,od_logistics,run_date_diff
0,2024-07-01,4,0,2,0,2024-07-12 - 2024-07-11
1,2024-06-01,0,0,0,0,2024-07-12 - 2024-07-11
2,2024-05-01,0,0,0,0,2024-07-12 - 2024-07-11
3,2024-04-01,0,0,0,0,2024-07-12 - 2024-07-11
4,2024-07-01,4,1,3,0,2024-07-15 - 2024-07-12
5,2024-06-01,0,0,0,0,2024-07-15 - 2024-07-12
6,2024-05-01,0,0,0,0,2024-07-15 - 2024-07-12
7,2024-04-01,0,0,0,0,2024-07-15 - 2024-07-12
8,2024-07-01,13660,2,1,0,2024-07-30 - 2024-07-15
9,2024-06-01,-78,0,0,0,2024-07-30 - 2024-07-15


In [33]:
df['month'] = pd.to_datetime(df['month']).dt.strftime('%b %Y')
data = df.to_dict(orient='records')

In [34]:
data

[{'month': 'Jul 2024',
  'OD_B2C': 4,
  'OD_B2B': 0,
  'OD_GV': 2,
  'od_logistics': 0,
  'run_date_diff': '2024-07-12 - 2024-07-11'},
 {'month': 'Jun 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  'OD_GV': 0,
  'od_logistics': 0,
  'run_date_diff': '2024-07-12 - 2024-07-11'},
 {'month': 'May 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  'OD_GV': 0,
  'od_logistics': 0,
  'run_date_diff': '2024-07-12 - 2024-07-11'},
 {'month': 'Apr 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  'OD_GV': 0,
  'od_logistics': 0,
  'run_date_diff': '2024-07-12 - 2024-07-11'},
 {'month': 'Jul 2024',
  'OD_B2C': 4,
  'OD_B2B': 1,
  'OD_GV': 3,
  'od_logistics': 0,
  'run_date_diff': '2024-07-15 - 2024-07-12'},
 {'month': 'Jun 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  'OD_GV': 0,
  'od_logistics': 0,
  'run_date_diff': '2024-07-15 - 2024-07-12'},
 {'month': 'May 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  'OD_GV': 0,
  'od_logistics': 0,
  'run_date_diff': '2024-07-15 - 2024-07-12'},
 {'month': 'Apr 2024',
  'OD_B2C': 0,
  'OD_B2B': 0,
  